## 2a. Evidence - Fairness QAS Measurements

Evidence collected in this section checks for the Fairness scenario defined in the previous step. Note that some functions will be loaded from external Python files.

The cell below must contain JSON data about this evidence that will be used to automatically populate the sample test catalog.

In [ ]:
{
    "tags": ["Computer Vision"],
    "quality_attribute": "Model Impartial to Photo Location",
    "description": "The model receives a picture taken at the garden and, regardless of the garden location, can correctly identify the correct flowers at least 90% of the time. Test data needs to include pictures of the flowers from the different gardens, grouped by the garden that the image was taken at. The quantity of the flower images should be representative of the garden population they are taken from. The total accuracy of the model across each garden population should be higher or equal to 0.9.",
    "inputs": "three garden populations, model results on Oxford garden data",
    "output": "accuracy across gardens",
}

### Initialize MLTE Context

MLTE contains a global context that manages the currently active _session_. Initializing the context tells MLTE how to store all of the artifacts that it produces. This import will also set up global constants related to folders and model to use.

In [ ]:
# Sets up context for the model being used, sets up constants related to folders and model data to be used.
from session import *

### Set up scenario test case

In [ ]:
from mlte.negotiation.artifact import NegotiationCard

card = NegotiationCard.load()
qa = 0
print(card.quality_scenarios[qa].identifier)
print(card.quality_scenarios[qa].quality)
print(
    card.quality_scenarios[qa].stimulus,
    "from ",
    card.quality_scenarios[qa].source,
    " during ",
    card.quality_scenarios[qa].environment,
    ". ",
    card.quality_scenarios[qa].response,
    card.quality_scenarios[qa].measure,
)

**A Specific test case generated from the scenario:**

**Data and Data Source:**	Test data needs to include pictures of the flowers from the different sections, grouped by the section that the image was taken at. The quantity of the flower images should be representative of the garden section population they are taken from.

**Measurement and Condition:**	The total accuracy of the model across each garden population should be higher or equal to 0.9.

**Context:**	Normal Operation

### Helper Functions
General functions and external imports.

In [ ]:
# General functions.

from utils import garden
import numpy as np


def load_data(data_folder: str):
    """Loads all garden data results and taxonomy categories."""
    df_results = garden.load_base_results(data_folder, "predictions_test.csv")
    df_results.head()

    # Load the taxonomic data and merge with results.
    df_info = garden.load_taxonomy(data_folder)
    df_results.rename(columns={"label": "Label"}, inplace=True)
    df_all = garden.merge_taxonomy_with_results(df_results, df_info)

    return df_info, df_all


def split_data(df_info, df_all, population_size=100):
    """Splits the data into 3 different populations to evaluate them."""
    df_gardenpop = df_info.copy()
    df_gardenpop["Population1"] = (
        np.around(
            np.random.dirichlet(np.ones(df_gardenpop.shape[0]), size=1)[0],
            decimals=3,
        )
        * population_size
    ).astype(int)
    df_gardenpop["Population2"] = (
        np.around(
            np.random.dirichlet(np.ones(df_gardenpop.shape[0]), size=1)[0],
            decimals=3,
        )
        * population_size
    ).astype(int)
    df_gardenpop["Population3"] = (
        np.around(
            np.random.dirichlet(np.ones(df_gardenpop.shape[0]), size=1)[0],
            decimals=3,
        )
        * population_size
    ).astype(int)
    # df_gardenpop
    print("Hello")  # df_gardenpop["Population2"])

    print(df_gardenpop["Population2"])
    # build populations from test data set that match the garden compositions
    from random import choices

    # build 3 gardens with populations of population_size.
    pop_names = ["Population1", "Population2", "Population3"]
    gardenpops = np.zeros((3, population_size), int)
    gardenmems = np.zeros((3, population_size), int)

    for j in range(population_size):
        for i in range(len(df_gardenpop)):
            my_flower = df_gardenpop.iloc[i]["Common Name"]

            for g in range(3):
                n_choices = df_gardenpop.iloc[i][pop_names[g]]
                my_choices = df_all[df_all["Common Name"] == my_flower][
                    "model correct"
                ].to_list()
                # print(f"{n_choices} {my_choices}")
                my_selection = choices(my_choices, k=n_choices)

                gardenpops[g][j] += sum(my_selection)
                gardenmems[g][j] += len(my_selection)

    gardenpops

    return gardenpops, gardenmems


def calculate_model_performance_acc(
    gardenpops, gardenmems, population_size=100
):
    """Get accucray of models across the garden populations"""
    gardenacc = np.zeros((3, population_size), float)
    for i in range(population_size):
        for g in range(3):
            gardenacc[g][i] = gardenpops[g][i] / gardenmems[g][i]
    gardenacc

    model_performance_acc = []
    for g in range(3):
        avg = round(np.average(gardenacc[g][:]), 3)
        std = round(np.std(gardenacc[g][:]), 3)
        min = round(np.amin(gardenacc[g][:]), 3)
        max = round(np.amax(gardenacc[g][:]), 3)
        model_performance_acc.append(round(avg, 3))

        print("%1d %1.3f %1.3f %1.3f %1.3f" % (g, avg, std, min, max))

    return model_performance_acc

In [ ]:
# Prepare the data. For this section, instead of executing the model, we will use CSV files containing the results of an already executed run of the model.
data = load_data(DATASETS_DIR)
print("Splitting Data")
# print(data)
split_data = split_data(data[0], data[1])

### Measurements

In this first example, we simply wrap the output from `accuracy_score` with a custom `Result` type to cope with the output of a third-party library that is not supported by a MLTE builtin.

In [ ]:
from mlte.evidence.types.array import Array
from mlte.measurement.external_measurement import ExternalMeasurement

# Evaluate accuracy, identifier has to be the same one defined in the TestSuite.
accuracy_measurement = ExternalMeasurement(
    "accuracy across gardens", Array, calculate_model_performance_acc
)
accuracy = accuracy_measurement.evaluate(split_data[0], split_data[1])

# Inspect value
print(accuracy)

# Save to artifact store
accuracy.save(force=True)